In [77]:
from kafka import KafkaConsumer
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm.auto as tqdm
import requests

In [51]:
def movie_feature(dff): #turn list of dictionary to list of strings with only the name
    df = dff.copy()
    df['genres'] = dff['genres'].apply( lambda x: [i['name'] for i in x])
    df['belongs_to_collection'] = dff['belongs_to_collection'].apply(lambda x:x['name'] if x != {} else None)
    df['production_companies'] = dff['production_companies'].apply(lambda x:[i['name'] for i in x])
    df['production_countries'] = dff['production_countries'].apply(lambda x:[i['name'] for i in x])
    df['spoken_languages'] = dff['spoken_languages'].apply(lambda x:[i['name'] for i in x])
    return df[df.columns.difference(['homepage','poster_path', ])]
# samp= movie_feature(sample)
# samp


,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,revenue,runtime,spoken_languages,status,title,tmdb_id,vote_average,vote_count
0,False,None,110000000,"[Drama, History, War, Action]",the+patriot+2000,tt0187393,en,The Patriot,After proving himself on the field of battle i...,15.960788,...,"[Germany, United States of America]",2000-06-28,215294342,165,[English],Released,The Patriot,2024,6.8,1130


In [78]:
#http://:8080/movie/batman_returns_1992 and http://<ip>:8080/user/100020
movie_list = ['the+patriot+2000', 'batman+returns+1992','blade+1998','boys+and+girls+2000', 'an+american+werewolf+in+paris+1997', 'nobody+else+but+you+2011']
not_found = []
for i, title in enumerate(movie_list):
    
    response = requests.get('http://128.2.204.215:8080/movie/'+title)
    movie_info = response.json()
    # print(movie_info)
    if movie_info.get('message') is None:
        if i==0:
            df = pd.read_json(json.dumps(movie_info), orient='index').T
            df = movie_feature(df)
        else:
            dff = pd.read_json(json.dumps(movie_info), orient='index').T
            dff = movie_feature(dff)
            df = pd.concat([df, dff], axis=0)
    else:
        not_found.append(title)
print(not_found)
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df)


[]


,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,tmdb_id,vote_average,vote_count
0,False,None,110000000,"[Drama, History, War, Action]",the+patriot+2000,tt0187393,en,The Patriot,After proving himself on the field of battle i...,15.960788,"[Centropolis Entertainment, Columbia Pictures ...","[Germany, United States of America]",2000-06-28,215294342,165,[English],Released,The Patriot,2024,6.8,1130
0,False,Batman Collection,80000000,"[Action, Fantasy]",batman+returns+1992,tt0103776,en,Batman Returns,"Having defeated the Joker, Batman now faces th...",15.001681,"[PolyGram Filmed Entertainment, Warner Bros.]","[United Kingdom, United States of America]",1992-06-19,280000000,126,[English],Released,Batman Returns,364,6.6,1706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,False,None,22000000,"[Horror, Comedy]",an+american+werewolf+in+paris+1997,tt0118604,en,An American Werewolf in Paris,An American man unwittingly gets involved with...,7.145784,"[Hollywood Pictures, Cometstone Pictures]","[France, United Kingdom, Luxembourg, Netherlan...",1997-12-25,26570463,105,"[English, Français]",Released,An American Werewolf in Paris,9406,5.3,154
0,False,None,0,"[Comedy, Crime, Mystery]",nobody+else+but+you+2011,tt1736636,en,Poupoupidou,"The ambiguous suicide of a local beauty, weath...",3.352027,"[France 2 Cinéma, Diaphana Productions]",[France],2011-01-12,0,102,[Français],Released,Nobody Else But You,53776,5.9,28


In [74]:
#http://:8080/movie/batman_returns_1992 and http://<ip>:8080/user/100020
user_list = ['99788', '43555','180321','190314', '106233']
not_found_user = []
for i, user in enumerate(user_list):
    
    response = requests.get('http://128.2.204.215:8080/user/'+user)
    user_info = response.json()
    if user_info.get('message') is None:
        if i==0:
            df_user = pd.read_json(json.dumps(user_info), orient='index').T
        else:
            dff = pd.read_json(json.dumps(user_info), orient='index').T
            df_user = pd.concat([df_user, dff], axis=0)
    else:
        not_found_user.append(user)
print(not_found)
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_user)


[]


,user_id,age,occupation,gender
0,99788,28,executive/managerial,F
0,43555,28,college/grad student,M
0,180321,34,executive/managerial,M
0,190314,37,programmer,M
0,106233,29,college/grad student,F
